In [2]:
!pip3 install pandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def analyze_market_impact(my_bid_prices, my_bid_sizes, show_detailed_calcs=False):
    """
    Analyze trading strategy against a market maker with 1-point impact per contract
    """
    # Initialize market maker state
    mm_mid = 50
    mm_spread = 2
    remaining_customer_size = 10
    my_fills = []
    mm_states = []
    
    # Convert inputs to sorted lists of tuples (price, size)
    my_orders = list(zip(my_bid_prices, my_bid_sizes))
    my_orders.sort(reverse=True)  # Sort by price descending
    
    # Process each price level
    detailed_steps = []
    for price, size in my_orders:
        while size > 0 and remaining_customer_size > 0:
            mm_bid = mm_mid - mm_spread/2
            mm_offer = mm_mid + mm_spread/2
            
            mm_states.append({
                'mid': mm_mid,
                'bid': mm_bid,
                'offer': mm_offer
            })
            
            # If our bid is above market maker's offer
            if price >= mm_offer:
                fill_size = min(size, remaining_customer_size)
                my_fills.append({
                    'price': mm_offer,
                    'size': fill_size
                })
                
                if show_detailed_calcs:
                    detailed_steps.append(
                        f"Filled {fill_size} @ {mm_offer:.2f} (MM offer) - "
                        f"Customer sells @ {mm_bid:.2f}"
                    )
                
                # Update states
                size -= fill_size
                remaining_customer_size -= fill_size
                mm_mid += fill_size  # Market moves up 1 point per contract
            else:
                break
    
    # Calculate P&L
    total_cost = sum(fill['price'] * fill['size'] for fill in my_fills)
    total_size = sum(fill['size'] for fill in my_fills)
    customer_sell_prices = []
    mm_mid = 50  # Reset to calculate customer fills
    
    for fill in my_fills:
        sell_price = mm_mid - mm_spread/2
        customer_sell_prices.extend([sell_price] * int(fill['size']))
        mm_mid += fill['size']
    
    total_revenue = sum(customer_sell_prices)
    pnl = total_revenue - total_cost if total_size > 0 else 0
    
    return {
        'fills': my_fills,
        'total_size': total_size,
        'total_cost': total_cost,
        'total_revenue': total_revenue,
        'pnl': pnl,
        'remaining_customer_size': remaining_customer_size,
        'detailed_steps': detailed_steps,
        'mm_states': mm_states
    }

def create_strategy_visualization(result):
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add market maker quotes over time
    mm_states = pd.DataFrame(result['mm_states'])
    x = list(range(len(mm_states)))
    
    fig.add_trace(
        go.Scatter(x=x, y=mm_states['offer'], name="MM Offer", line=dict(color='red')),
        secondary_y=False,
    )
    
    fig.add_trace(
        go.Scatter(x=x, y=mm_states['bid'], name="MM Bid", line=dict(color='green')),
        secondary_y=False,
    )
    
    fig.add_trace(
        go.Scatter(x=x, y=mm_states['mid'], name="MM Mid", line=dict(color='gray', dash='dash')),
        secondary_y=False,
    )
    
    # Update layout
    fig.update_layout(
        title='Market Maker Quotes Progression',
        xaxis_title='Time Step',
        yaxis_title='Price',
        height=500
    )
    
    return fig

def create_interactive_analysis():
    style = {'description_width': 'initial'}
    
    price1 = widgets.FloatSlider(value=51.0, min=0, max=100.0, step=0.1, 
                                description='Order 1 Price:', style=style)
    size1 = widgets.FloatSlider(value=5.0, min=0, max=10.0, step=0.1, 
                               description='Order 1 Size:', style=style)
    
    price2 = widgets.FloatSlider(value=52.0, min=0, max=100.0, step=0.1, 
                                description='Order 2 Price:', style=style)
    size2 = widgets.FloatSlider(value=5.0, min=0, max=10.0, step=0.1, 
                               description='Order 2 Size:', style=style)
    
    show_calcs = widgets.Checkbox(value=False, description='Show detailed calculations')
    
    def analyze(p1, s1, p2, s2, show_detailed):
        clear_output(wait=True)
        display(widgets.VBox([price1, size1, price2, size2, show_calcs]))
        
        result = analyze_market_impact(
            my_bid_prices=[p1, p2],
            my_bid_sizes=[s1, s2],
            show_detailed_calcs=show_detailed
        )
        
        # Display results
        print("\n=== Results ===")
        print(f"Total fills: {result['total_size']} contracts")
        print(f"Average fill price: ${result['total_cost']/result['total_size']:.2f}" if result['total_size'] > 0 else "No fills")
        print(f"P&L: ${result['pnl']:.2f}")
        print(f"Remaining customer size: {result['remaining_customer_size']} contracts")
        
        if show_detailed and result['detailed_steps']:
            print("\n=== Detailed Steps ===")
            for step in result['detailed_steps']:
                print(step)
        
        # Display visualization
        fig = create_strategy_visualization(result)
        fig.show()
    
    widgets.interactive_output(analyze, {
        'p1': price1, 's1': size1, 
        'p2': price2, 's2': size2,
        'show_detailed': show_calcs
    })
    
    return widgets.VBox([price1, size1, price2, size2, show_calcs])

# Create and display the interactive tool
interactive_analysis = create_interactive_analysis()
display(interactive_analysis)

ModuleNotFoundError: No module named 'pandas'